In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Step 1: Load and Process Player Per Game Data
print("--- Step 1: Loading and Processing 'Player Per Game.csv' ---")
try:
    player_df = pd.read_csv('Player Per Game.csv')
    print("Successfully loaded 'Player Per Game.csv'.")
    print("Initial 'Player Per Game.csv' head:")
    print(player_df.head())
    print("\nInitial 'Player Per Game.csv' info:")
    player_df.info()
except FileNotFoundError:
    print("Error: 'Player Per Game.csv' not found. Please upload this file to proceed.")
    exit()

target_seasons = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df_filtered = player_df[player_df['season'].isin(target_seasons)].copy()
print(f"\nShape after season filtering: {df_filtered.shape}")

essential_cols = [
    'mp_per_game', 'pts_per_game', 'trb_per_game', 'ast_per_game',
    'stl_per_game', 'blk_per_game', 'tov_per_game', 'pf_per_game',
    'fg_percent', 'x3p_percent', 'ft_percent', 'e_fg_percent',
    'orb_per_game', 'drb_per_game'
]
essential_cols_exist = [col for col in essential_cols if col in df_filtered.columns]
df_cleaned = df_filtered.dropna(subset=essential_cols_exist).copy()
df_cleaned.drop_duplicates(subset=['player_id', 'season', 'team'], inplace=True)
min_games = 10
min_minutes_per_game = 5
df_qualified = df_cleaned[
    (df_cleaned['g'] >= min_games) &
    (df_cleaned['mp_per_game'] >= min_minutes_per_game)
].copy()
features_for_clustering = [col for col in essential_cols if col in df_qualified.columns]
X = df_qualified[features_for_clustering].copy()
player_identifiers = df_qualified[['player', 'season', 'team', 'pos']].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

optimal_k = 4
kmeans_model = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
clusters = kmeans_model.fit_predict(X_scaled)
player_archetypes = player_identifiers.copy()
player_archetypes['cluster'] = clusters

cluster_name_mapping = {
    0: "Efficient Role Player",
    1: "Offensive Engine",
    2: "3pt Specialist",
    3: "Interior Force"
}
player_archetypes['archetype_name'] = player_archetypes['cluster'].map(cluster_name_mapping)

print("\nPlayer archetypes DataFrame ready for further analysis.")
print(player_archetypes.head())
print(f"Unique teams in player_archetypes: {player_archetypes['team'].unique()}")
print(f"Unique seasons in player_archetypes: {player_archetypes['season'].unique()}")


# Step 2: Load and Process Team Summaries Data to create team_win_percentages.csv 
print("\n--- Step 2: Loading and Processing 'Team Summaries.csv' ---")
try:
    ts = pd.read_csv('Team Summaries.csv')
    print("Successfully loaded 'Team Summaries.csv'.")
    print("Initial 'ts' DataFrame head:")
    print(ts.head())
    print("\nInitial 'ts' DataFrame info:")
    ts.info()
except FileNotFoundError:
    print("Error: 'Team Summaries.csv' not found. Please upload this file to proceed.")
    exit()

team_seasons_filter = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
df_teams = ts[ts['season'].isin(team_seasons_filter)].copy()

columns_to_drop_ts = ['age', 'mov', 'sos', 'srs', 'o_rtg', 'd_rtg', 'n_rtg', 'pace', 'f_tr', 'x3p_ar', 'ts_percent',
                      'e_fg_percent', 'tov_percent', 'orb_percent', 'drb_percent', 'opp_ft_fga', 'arena', 'attend',
                      'attend_g', 'ft_fga', 'opp_e_fg_percent', 'opp_tov_percent']

columns_to_drop_ts_exist = [col for col in columns_to_drop_ts if col in df_teams.columns]
df_teams = df_teams.drop(columns=columns_to_drop_ts_exist)

if 'w' in df_teams.columns:
    df_teams['win_percentage'] = df_teams['w']/82
else:
    print("Warning: 'w' (wins) column not found in df_teams. Cannot calculate 'win_percentage'.")

df_teams['season'] = df_teams['season'].astype(int)
df_teams['team'] = df_teams['team'].astype(str).str.strip()

# Create a mapping from full team names to abbreviations
team_name_mapping = {
    'Atlanta Hawks': 'ATL', 'Boston Celtics': 'BOS', 'Brooklyn Nets': 'BRK',
    'Chicago Bulls': 'CHI', 'Charlotte Hornets': 'CHO', 'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'
}

# Apply the mapping to the 'team' column
df_teams['team'] = df_teams['team'].apply(lambda x: team_name_mapping.get(x, x))
print("\nTeam names in df_teams standardized to abbreviations.")

df_teams['win_percentage'] = pd.to_numeric(df_teams['win_percentage'], errors='coerce')
df_teams.dropna(subset=['win_percentage'], inplace=True)

print("\nProcessed 'df_teams' head (ready for saving as CSV):")
print(df_teams.head())
print(f"Unique teams in df_teams (after standardization): {df_teams['team'].unique()}")
print(f"Unique seasons in df_teams: {df_teams['season'].unique()}")


# Save the processed team data to a CSV file
output_csv_filename = 'team_win_percentages.csv'
df_teams.to_csv(output_csv_filename, index=False)
print(f"\n'{output_csv_filename}' created successfully.")


# Step 3: Integrate Archetype Composition with Team Winning Percentages
print("\n--- Step 3: Calculating Team Archetype Composition ---")

team_composition = player_archetypes.groupby(['team', 'season', 'archetype_name']).size().unstack(fill_value=0)
team_composition['total_qualified_players'] = team_composition.sum(axis=1)

print("Team archetype composition calculated:")
print(team_composition.head())
print(f"Columns in team_composition: {team_composition.columns.tolist()}")


# Load the newly created team_win_percentages.csv
print(f"\n--- Step 3.1: Loading '{output_csv_filename}' for merging ---")
team_wins_df_for_merge = pd.read_csv(output_csv_filename)
print("Team winning percentages loaded successfully for merge.")
print("Head of team_wins_df_for_merge:")
print(team_wins_df_for_merge.head())
print("Info of team_wins_df_for_merge:")
team_wins_df_for_merge.info()

# Ensure consistency in team and season columns for merging
team_wins_df_for_merge['season'] = team_wins_df_for_merge['season'].astype(int)
team_wins_df_for_merge['team'] = team_wins_df_for_merge['team'].astype(str).str.strip()

print("\n--- Step 3.2: Merging Archetype Composition with Winning Percentages ---")

team_analysis_df = pd.merge(
    team_composition.reset_index(),
    team_wins_df_for_merge[['team', 'season', 'win_percentage']], # Select only necessary columns to avoid duplicates
    on=['team', 'season'],
    how='inner'
)

if team_analysis_df.empty:
    print("Error: Merged DataFrame is empty. This indicates a mismatch in 'team' or 'season' between the two datasets.")
    print(f"Unique teams in player_archetypes: {player_archetypes['team'].unique()}")
    print(f"Unique seasons in player_archetypes: {player_archetypes['season'].unique()}")
    print(f"Unique teams in team_wins_df_for_merge: {team_wins_df_for_merge['team'].unique()}")
    print(f"Unique seasons in team_wins_df_for_merge: {team_wins_df_for_merge['season'].unique()}")
else:
    print("Merged data for analysis:")
    print(team_analysis_df.head())
    print(f"Columns in team_analysis_df: {team_analysis_df.columns.tolist()}")

    # Step 4: Analyze How Specific Clusters Influence Team Winning Percentage 
    print("\n--- Step 4: Analyzing Cluster Influence on Winning Percentage ---")

    archetype_cols = list(cluster_name_mapping.values())
    print(f"Archetype columns being used for correlation: {archetype_cols}")

    print("\nCorrelation of Archetype Counts with Winning Percentage:")
    correlations = team_analysis_df[archetype_cols].corrwith(team_analysis_df['win_percentage'])
    print(correlations)

    # Visualize average win percentage by varying count of a specific archetype
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Offensive Engine', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage by Number of Offensive Engines on Team')
    plt.xlabel('Number of Offensive Engines')
    plt.ylabel('Win Percentage')
    plt.grid(axis='y')
    plt.savefig('offensive_engine_win_percentage.png')
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Interior Force', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage by Number of Interior Forces on Team')
    plt.xlabel('Number of Interior Forces')
    plt.ylabel('Win Percentage')
    plt.grid(axis='y')
    plt.savefig('interior_force_win_percentage.png')
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='3pt Specialist', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage by Number of 3pt Specialists on Team')
    plt.xlabel('Number of 3pt Specialists')
    plt.ylabel('Win Percentage')
    plt.grid(axis='y')
    plt.savefig('3pt_specialist_win_percentage.png')
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Efficient Role Player', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage by Number of Efficient Role Players on Team')
    plt.xlabel('Number of Efficient Role Players')
    plt.ylabel('Win Percentage')
    plt.grid(axis='y')
    plt.savefig('efficient_role_player_win_percentage.png')
    plt.close()


    # Step 5: Analyze What Combination of Clusters Work Best for Team Win Percentage ---
    print("\n--- Step 5: Analyzing Best Combinations of Clusters ---")

    # Approach A: Examine top-performing teams' archetype compositions
    print("\nTop 5 Teams by Win Percentage and their Archetype Composition:")
    top_teams = team_analysis_df.sort_values(by='win_percentage', ascending=False).head(5)
    print(top_teams[['team', 'season', 'win_percentage', 'total_qualified_players'] + archetype_cols])

    # Approach B: Analyze ratios of archetypes
    # Example: ratio of Offensive Engines to Interior Force
    team_analysis_df['OE_IF_ratio'] = team_analysis_df['Offensive Engine'] / (team_analysis_df['Interior Force'] + 1e-6)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='OE_IF_ratio', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage vs. Offensive Engine to Interior Force Ratio')
    plt.xlabel('Offensive Engine / Interior Force Ratio')
    plt.ylabel('Win Percentage')
    plt.xscale('log')
    plt.grid(True)
    plt.savefig('oe_if_ratio_win_percentage.png')
    plt.close()

    # Example: ratio of 3pt Specialist to Interior Force
    team_analysis_df['3PTS_IF_ratio'] = team_analysis_df['3pt Specialist'] / (team_analysis_df['Interior Force'] + 1e-6)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='3PTS_IF_ratio', y='win_percentage', data=team_analysis_df)
    plt.title('Win Percentage vs. 3pt Specialist to Interior Force Ratio')
    plt.xlabel('3pt Specialist / Interior Force Ratio')
    plt.ylabel('Win Percentage')
    plt.xscale('log')
    plt.grid(True)
    plt.savefig('3pts_if_ratio_win_percentage.png')
    plt.close()

    print("\nAnalysis complete. The correlations and visualizations provide insights into how archetype composition relates to team winning percentage.")

--- Step 1: Loading and Processing 'Player Per Game.csv' ---
Successfully loaded 'Player Per Game.csv'.
Initial 'Player Per Game.csv' head:
   season   lg            player  player_id   age team pos   g    gs  \
0    2025  NBA  Precious Achiuwa  achiupr01  25.0  NYK   C  57  10.0   
1    2025  NBA      Steven Adams  adamsst01  31.0  HOU   C  58   3.0   
2    2025  NBA       Bam Adebayo  adebaba01  27.0  MIA   C  78  78.0   
3    2025  NBA      Ochai Agbaji  agbajoc01  24.0  TOR  SG  64  45.0   
4    2025  NBA      Santi Aldama  aldamsa01  24.0  MEM  PF  65  16.0   

   mp_per_game  ...  ft_percent  orb_per_game  drb_per_game  trb_per_game  \
0         20.5  ...       0.594           1.8           3.8           5.6   
1         13.7  ...       0.462           2.9           2.8           5.6   
2         34.3  ...       0.765           2.4           7.2           9.6   
3         27.2  ...       0.708           1.0           2.8           3.8   
4         25.5  ...       0.691           